In [34]:
# Imports
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import stopwords
from datetime import datetime
pd.options.display.max_colwidth = 100000

In [2]:
# Read pickle file with topics and augmented text
df4 = pd.read_pickle("data/allData.pkl")
decompTopic = 5

In [3]:
for x in df4[df4['TOPIC'] == decompTopic]['TEXT'].head(20):
    print(x)

airspace volk south military operations area active foot up to but not including flight level
airspace volk east military operations area active foot up to but not including flight level
airspace falls military operations area active foot up to but not including flight level
airspace falls military operations area active foot up to but not including flight level
airspace volk west military operations area active foot up to but not including flight level
airspace jackal military operations area active foot up to but not including flight level
airspace jackal low military operations area active foot foot
airspace columbus military operations area active foot up to but not including flight level
airspace columbus military operations area active foot up to but not including flight level
airspace abel east military operations area active foot foot
obstruct tower light airport surveillance radar north or northern latitude west or western longitude nautical mile southsouth east advance bounda

In [4]:
corpus = df4[df4['TOPIC'] == decompTopic]['TEXT'].values

count_vect = CountVectorizer(stop_words=stopwords.words('english'), lowercase=True)
x_counts = count_vect.fit_transform(corpus)
x_counts.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [5]:
tfidf_transformer = TfidfTransformer()
x_tfidf = tfidf_transformer.fit_transform(x_counts)

In [6]:
dimension = 8
sample_size = 100000

np.random.seed(232323)
sample = x_tfidf[np.random.choice(x_tfidf.shape[0], sample_size, replace=False, ), :]

lda = LDA(n_components = dimension)
lda_array = lda.fit_transform(sample)

In [7]:
topic_vec = lda.transform(x_tfidf)

df4['SUBTOPIC'] = None

df4.loc[df4['TOPIC'] == 5, 'SUBTOPIC'] = np.argmax(topic_vec, axis = 1)

In [8]:
import pickle

#df4.to_pickle("data/allData.pkl")

In [9]:
def ntopwlst(model, features, ntopwords):
    '''create a list of the top topc words'''
    output = {}
    for topic_idx, topic in enumerate(model.components_):
        tmp = []
        tmp += [features[i] for i in topic.argsort()[:-ntopwords - 1:-1]] 
        output[topic_idx] = tmp
    return output

ntopwords = 20
features = count_vect.get_feature_names_out()
topwds = ntopwlst(lda, features, ntopwords)
topwds

{0: ['operations',
  'military',
  'including',
  'flight',
  'level',
  'area',
  'airspace',
  'foot',
  'active',
  'radius',
  'hart',
  'juniper',
  'system',
  'agl',
  'volk',
  'blue',
  'low',
  'absolute',
  'danger',
  'temperature'],
 1: ['eastern',
  'ausots',
  'southern',
  'south',
  'east',
  'tracks',
  'flex',
  'group',
  'latitude',
  'longitude',
  'closed',
  'information',
  'blue',
  'service',
  'taxiway',
  'time',
  'available',
  'altostratus',
  'defined',
  'return'],
 2: ['identification',
  'followed',
  'restricted',
  'area',
  'surface',
  'airspace',
  'active',
  'foot',
  'level',
  'flight',
  'absolute',
  'temperature',
  'blue',
  'celsius',
  'night',
  'training',
  'vision',
  'goggle',
  'danger',
  'activated'],
 3: ['foot',
  'latitude',
  'west',
  'northern',
  'north',
  'obstruct',
  'nautical',
  'mile',
  'tower',
  'radar',
  'light',
  'airport',
  'western',
  'longitude',
  'surveillance',
  'agl',
  'service',
  'unknown',
  '

In [10]:
import pyLDAvis
import pyLDAvis.sklearn
import warnings

warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

In [11]:
pyLDAvis.sklearn.prepare(lda, x_tfidf, count_vect)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.030513  0.285608       1        1  37.772615
2      0.203308 -0.137977       2        1  14.598001
0      0.220490 -0.042259       3        1  13.071062
6     -0.211540 -0.239137       4        1   7.858319
5      0.118679  0.087631       5        1   7.629632
7      0.222407 -0.005275       6        1   7.421842
1     -0.260402 -0.117440       7        1   7.091206
4     -0.262430  0.168850       8        1   4.557324, topic_info=                Term          Freq         Total Category  logprob  loglift
7156         western  27747.000000  27747.000000  Default  30.0000  30.0000
7154            west  31290.000000  31290.000000  Default  29.0000  29.0000
2532        followed  16790.000000  16790.000000  Default  28.0000  28.0000
3190  identification  16791.000000  16791.000000  Default  27.0000  27.0000
46            active  34268.000000  34268.000000  Default  26.0000  26.0000
...              ...           ...           ...      ...      ...      ...
5062              pb    157.009606    185.916559   Topic8  -5.8087   2.9194
1121         central    223.632096    574.784533   Topic8  -5.4550   2.1444
2043            east    367.375076  14694.932824   Topic8  -4.9586  -0.6004
2536            foot    360.519770  40969.831920   Topic8  -4.9774  -1.6446
147              agl    288.477794  19651.259808   Topic8  -5.2004  -1.1328

[478 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
8         2  0.988442  abcd
8         4  0.003306  abcd
8         7  0.003306  abcd
8         8  0.003306  abcd
17        3  0.992748  abel
...     ...       ...   ...
7355      3  0.521051  zone
7355      4  0.001160  zone
7355      5  0.416609  zone
7355      7  0.051061  zone
7355      8  0.001160  zone

[1383 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 7, 6, 8, 2, 5])

In [12]:
# Make MIN_ALT and MAX_ALT numeric
df4['MIN_ALT'] = df4['MIN_ALT'].replace('MSL', 0)
df4['MAX_ALT'] = df4['MAX_ALT'].replace('MSL', 0)
df4['MIN_ALT'] = pd.to_numeric(df4['MIN_ALT'], errors='coerce')
df4['MAX_ALT'] = pd.to_numeric(df4['MAX_ALT'], errors='coerce')

# Make POSSIBLE_START_DATE, POSSIBLE_END_DATE, ISSUE_DATE, and CANCELED_DATE datetime format
df4['POSSIBLE_START_DATE'] = pd.to_datetime(df4['POSSIBLE_START_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['POSSIBLE_END_DATE'] = pd.to_datetime(df4['POSSIBLE_END_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['ISSUE_DATE'] = pd.to_datetime(df4['ISSUE_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['CANCELED_DATE'] = pd.to_datetime(df4['CANCELED_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')

# Replace missing POSSIBLE_START_DATE with CANCELED_DATE and vise versa
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_END_DATE']
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'CANCELED_DATE']

# Get mode of all NOTAM durations
est_days = (df4['POSSIBLE_END_DATE'] - df4['POSSIBLE_START_DATE']).mode()

# Fix remaining missing values of CANCELED_DATE and POSSIBLE_END_DATE by adding the mode to the POSSIBLE_START_DATE
df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_START_DATE'] + pd.to_timedelta(est_days.dt.days[0], unit='D')

# Drop NOTAMs without POSSIBLE_END_DATE and POSSIBLE_START_DATE
df4 = df4.dropna( how='all', subset=['POSSIBLE_END_DATE', 'POSSIBLE_START_DATE'])

In [43]:
# One off testing and tuning

#time = '10/05/2017  08:00:00 AM' #349
#time = '2/20/2016  7:40:00 PM' #230
#time = '4/22/2020  7:30:30 PM' #514
#time = '4/2/2018  8:30:38 PM' #391
#time = '11/16/2017  2:52:00 AM' #364
#time = '3/1/2017  5:45:00 PM' #300
#time = '5/11/2018  8:14:00 PM' #400
time = '12/5/2018  6:16:16 PM' #438

date_time_obj = datetime.strptime(time, '%m/%d/%Y %H:%M:%S %p')

In [44]:
# Date filter
filterA = df4[(df4['POSSIBLE_START_DATE'] <= time) & (df4['POSSIBLE_END_DATE'] >= time)]

In [45]:
# Altitude filter
filterB = filterA[filterA['MAX_ALT'] >= 50000 | filterA['MAX_ALT'].isna()]

In [46]:
# Keyword filter
filterC = filterB[(filterB['TEXT'].str.contains(r'(?:\s|^)rocket(?:\s|$)') == True) | 
                      (filterB['TEXT'].str.contains(r'(?:\s|^)space(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)launch(?:\s|$)') == True) |
                      #(filterB['TEXT'].str.contains(r'(?:\s|^)satellite(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)missile(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)canaveral(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)kennedy(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)nasa(?:\s|$)') == True) |
                      (filterB['TEXT'].str.contains(r'(?:\s|^)unlimited(?:\s|$)') == True)]

In [47]:
# Read FIR Codes
df = pd.read_csv('data/NA_FIR_Codes.csv')

# Read US Airport Codes
#df2 = pd.read_csv('data/AirportData_Clean_20210629_Geocoded.csv')

In [48]:
# FIR Code filter
filterD = filterC[filterC['LOCATION_CODE'].isin(df['FIR']) | filterC['LOCATION_CODE'].isna()]

# US Airport Code filter
#filterD = filterC[filterC['LOCATION_CODE'].isin(df['FIR']) | filterC['LOCATION_CODE'].isna() | filterC['LOCATION_CODE'].isin(df2['locid'])]

In [49]:
# Print out
filterD[['TOPIC','SUBTOPIC','NOTAM_REC_ID', 'TEXT']]

TOPIC SUBTOPIC  NOTAM_REC_ID  \
938285       7     None        938870   
1146236      3     None       1146924   
1146260      3     None       1146949   
1146264      3     None       1146953   
1146265      3     None       1146954   
1146295      3     None       1146984   
1146299      3     None       1146988   
1146301      3     None       1146990   
1146340      3     None       1147029   
1146363      3     None       1147052   
1146433      3     None       1147122   
1146435      3     None       1147124   
1150652      2     None       1151342   
1150654      2     None       1151344   
1151006      2     None       1151696   
1151275      5        2       1151965   
1151277      5        2       1151967   
1151546      2     None       1152236   
1151549      2     None       1152239   
1155583      5        0       1156273   
1155585      5        0       1156275   
1155586      5        0       1156276   
1155587      5        0       1156277   
1155588      5        0       1156278   
1155589      5        0       1156279   
1155595      5        0       1156285   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [59]:
# Topic filter
filterE = filterD[(filterD['TOPIC'] == 3) | (filterD['SUBTOPIC'] == 2)]

filterE[['TOPIC','SUBTOPIC','NOTAM_REC_ID', 'TEXT']]

TOPIC SUBTOPIC  NOTAM_REC_ID  \
1146236      3     None       1146924   
1146260      3     None       1146949   
1146264      3     None       1146953   
1146265      3     None       1146954   
1146295      3     None       1146984   
1146299      3     None       1146988   
1146301      3     None       1146990   
1146340      3     None       1147029   
1146363      3     None       1147052   
1146433      3     None       1147122   
1146435      3     None       1147124   
1151275      5        2       1151965   
1151277      5        2       1151967   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          